# 1. SETTINGS

In [74]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr
import random

In [75]:
# garbage collection
import gc
gc.enable()

In [76]:
# warnings
import warnings
warnings.filterwarnings("ignore")

In [77]:
# pandas options
pd.set_option("display.max_columns", None)

# 2. IMPORT

In [78]:
# data name (used for saving files too)
data_name = "data_v4_0_60_under"

In [79]:
# import datasets
data = pd.read_csv("../data/prepared/" + str(data_name) + ".csv", compression = "gzip")

In [80]:
# data name (used for saving files too)
data_name = "data_v4_0_60_under_wlp_lm_bm"

In [81]:
# drop high recency from train
#data = data[(data.Week == 121) | (data.Recency2 < 27)]
data = data[(data.Week == 121) | (data.Recency1 < data.Recency1.max())]
data.shape

(2668912, 45)

# 3. PREPARATIONS

### ADD MORE FEATURES

In [82]:
# compute BuySell dummy
data["Buy"] = 0
data["Buy"][data.BuySell == "Buy"] = 1
print(data.shape)

(2668912, 45)


In [83]:
### RF Ratios

#data["R12_ratio"] = data.Recency1 / data.Recency2
#data["R13_ratio"] = data.Recency1 / data.Recency3
#data["R14_ratio"] = data.Recency1 / data.Recency4
#data["R56_ratio"] = data.Recency5 / data.Recency6

#data["F12_ratio"] = data.Frequency1 / data.Frequency2
#data["F13_ratio"] = data.Frequency1 / data.Frequency3
#data["F14_ratio"] = data.Frequency1 / data.Frequency4
#data["F56_ratio"] = data.Frequency5 / data.Frequency6

#data.shape

In [84]:
# add last week sum (CUST)
tmp = data.sort_values(by = ["Week"], ascending = True).groupby(["CustomerIdx", "Week"], as_index = True)
tmp = tmp.CustomerInterest.sum().reset_index()
tmp.columns = ["CustomerIdx", "Week", "LastWeekCustSum"]
tmp["Week"] = tmp.Week + 1
data = data.merge(tmp, how = "left", on = ["CustomerIdx", "Week"])
print(data.shape)

# add last week sum (BOND)
tmp = data.sort_values(by = ["Week"], ascending = True).groupby(["IsinIdx", "Week"], as_index = True)
tmp = tmp.CustomerInterest.sum().reset_index()
tmp.columns = ["IsinIdx", "Week", "LastWeekBondSum"]
tmp["Week"] = tmp.Week + 1
data = data.merge(tmp, how = "left", on = ["IsinIdx", "Week"])
print(data.shape)

(2668912, 46)
(2668912, 47)


In [85]:
### ADD PRICE AND NOTIONAL

tmp = pd.read_csv("../data/prepared/price_notional.csv", compression = "gzip")

# add last week price sum (CUST)
tmp2 = tmp.sort_values(by = ["Week"], ascending = True).groupby(["CustomerIdx", "Week"], as_index = True)
tmp2 = tmp2.Price.sum().reset_index()
tmp2.columns = ["CustomerIdx", "Week", "LastWeekCustPriceSum"]
tmp2["Week"] = tmp2.Week + 1
data = data.merge(tmp2, how = "left", on = ["CustomerIdx", "Week"])
print(data.shape)

# add last week notional sum (CUST)
tmp2 = tmp.sort_values(by = ["Week"], ascending = True).groupby(["CustomerIdx", "Week"], as_index = True)
tmp2 = tmp2.NotionalEUR.sum().reset_index()
tmp2.columns = ["CustomerIdx", "Week", "LastWeekCustNotionalSum"]
tmp2["Week"] = tmp2.Week + 1
data = data.merge(tmp2, how = "left", on = ["CustomerIdx", "Week"])
print(data.shape)

(2668912, 48)
(2668912, 49)


In [86]:
### ADD CUSTOMER INDICATORS

#cust  = pd.read_csv("../data/raw/Customer.csv")
#cust.head()

# create dummies for customers
#cust_dummies = pd.concat([cust.CustomerIdx, pd.get_dummies(cust.Subsector)], axis = 1)

# merge customer dummies
#data = data.merge(cust_dummies, on = "CustomerIdx", how = "left")
#print(data.shape)

In [87]:
### ADD BOND FEATURES [V1]

# load bond data
#bond_stat = pd.read_csv("../data/prepared/bondstat1_AK")

# merge bond data
#data = data.merge(bond_stat[["IsinIdx", "t", "timeOnMarket", "percentoflifeleft"]], how = "left", on = "IsinIdx")
#print(data.shape)

In [88]:
### ADD NEW BOND FEATURES [V2]

# load bond data
#bond_stat = pd.read_csv("../data/prepared/bondstat2_AK")

# merge bond data
#data = data.merge(bond_stat[["IsinIdx", 'ymeansq', 'ymeancube', 'zmeansq', 'zmeancube',
#                             'years_tomaturity', 'count', 'ratingspread', 'ratingZspread']], 
#                  how = "left", on = "IsinIdx")
#print(data.shape)

In [89]:
### ADD BOND FEATURES [V3]

# load bond data
#bond_stat = pd.read_csv("../data/prepared/bondstat3_AK")
#bond_stat.head()

# merge bond data
#data = data.merge(bond_stat[["IsinIdx", 'predyield', 'predprice']], how = "left", on = "IsinIdx")
#print(data.shape)

In [90]:
### BOND-SPECIFIC FINANCIAL INDICATORS

#bond  = pd.read_csv("../data/raw/Isin.csv")

# create bond dummies
#bond_dummies = pd.concat([bond.IsinIdx,
#                          pd.get_dummies(bond.IndustrySector), 
#                          pd.get_dummies(bond.Region),
#                          pd.get_dummies(bond.MarketIssue), 
#                          pd.get_dummies(bond.CouponType)], axis = 1)

# merge bond dummies
#data = data.merge(bond_dummies, on = "IsinIdx", how = "left")
#print(data.shape)

In [91]:
### ADD NEW NIKITA FEATURES

# import data
#cust    = pd.read_csv("../data/prepared/cust_nik.csv",   compression = "gzip")
#bond    = pd.read_csv("../data/prepared/bond_nik.csv",   compression = "gzip")
#market  = pd.read_csv("../data/prepared/market_nik.csv", compression = "gzip")

#cust = cust[["CustomerIdx", "Region"]]
#bond = bond[["IsinIdx", "ActualMaturityDateKey", "IssueDateKey"]]

# merge data
#print(data.shape)
#data = data.merge(cust, on = "CustomerIdx", how = "left")
#print(data.shape)
#data = data.merge(bond, on = "IsinIdx", how = "left")
#print(data.shape)
#data = data.merge(market, on = ["IsinIdx", "Week"], how = "left")
#print(data.shape)

# compute day differences
#data["ActualMaturityDateKey"] = data["ActualMaturityDateKey"] - data["Week"]
#data["IssueDateKey"]          = data["IssueDateKey"] - data["Week"]

In [92]:
### ADD LAST MONTH SUMS

# merge month number
tmp = pd.read_csv("../data/raw/Trade.csv")[["TradeDateKey"]]
tmp = tmp.drop_duplicates()
tmp["TradeDateKey"] = pd.to_datetime(tmp["TradeDateKey"], format = '%Y%m%d')
tmp["Week"] = (tmp.TradeDateKey.dt.year - 2016) * 52 + (tmp.TradeDateKey.dt.week)
tmp["CumMonth"] = (tmp.TradeDateKey.dt.year - 2016) * 12 + (tmp.TradeDateKey.dt.month)
tmp = tmp[["Week", "CumMonth"]]
tmp = tmp.drop_duplicates()
tmp = tmp.groupby("Week").CumMonth.min().reset_index()
data = data.merge(tmp, how = "left", on = "Week")
data["CumMonth"][data.Week == 121] = 28

# add last month mean (CUST)
tmp = data.sort_values(by = ["CumMonth"], ascending = True).groupby(["CustomerIdx", "CumMonth"], as_index = True)
tmp = tmp.CustomerInterest.sum().reset_index()
tmp.columns = ["CustomerIdx", "CumMonth", "LastMonthCustSum"]
tmp["CumMonth"] = tmp.CumMonth + 1
data = data.merge(tmp, how = "left", on = ["CustomerIdx", "CumMonth"])
print(data.shape)

# add last month mean (BOND)
tmp = data.sort_values(by = ["CumMonth"], ascending = True).groupby(["IsinIdx", "CumMonth"], as_index = True)
tmp = tmp.CustomerInterest.sum().reset_index()
tmp.columns = ["IsinIdx", "CumMonth", "LastMonthBondSum"]
tmp["CumMonth"] = tmp.CumMonth + 1
data = data.merge(tmp, how = "left", on = ["IsinIdx", "CumMonth"])
print(data.shape)

# drop month
del data["CumMonth"]

(2668912, 51)
(2668912, 52)


In [93]:
### ADD BOND MATURITY DATA

# import
bond = pd.read_csv("../data/prepared/data_bond_v1.csv", compression = "gzip")
bond = bond[['IsinIdx', 'MaturityWeek', 'IssueWeek']]

# merge
data = data.merge(bond, on = "IsinIdx", how = "left")

# compute week differences
data["MaturityWeek"]    = data.MaturityWeek - data.Week
data["IssueWeek"]       = data.Week - data.IssueWeek
data["MaturityPercent"] = (data.Week - data.IssueWeek) / (data.MaturityWeek - data.IssueWeek)
print(data.shape)

(2668912, 54)


In [94]:
### MERGE WEEKLY MACRO VARIABLES

# import
#fx1   = pd.read_csv("../data/prepared/data_fx1_v1.csv",  compression = "gzip")
#fx2   = pd.read_csv("../data/prepared/data_fx2_v1.csv",  compression = "gzip")
#ind1  = pd.read_csv("../data/prepared/data_ind1_v1.csv", compression = "gzip")
#ind2  = pd.read_csv("../data/prepared/data_ind2_v1.csv", compression = "gzip")

# merge currency
#bond = pd.read_csv("../data/raw/Isin.csv")
#data = data.merge(bond[["IsinIdx", "Currency"]], on = "IsinIdx")

# merge macro data
#data = data.merge(fx1,  how = 'left', on = ["Week",  "Currency"])
#print(data.shape)
#data = data.merge(fx2,  how = 'left', on = ["Month", "Currency"])
#print(data.shape)
#data = data.merge(ind1, how = 'left', on = "Week")
#print(data.shape)
#data = data.merge(ind2, how = 'left', on = "Month")
#print(data.shape)

# there are a few currencies for which we don't have info, e.g. CNH
#data[fx1.columns] = data[fx1.columns].fillna(0)
#data[fx2.columns] = data[fx2.columns].fillna(0)

# remove currency
#data.drop("Currency", axis = 1, inplace = True)
#print(data.shape)

In [95]:
### ADD NEW WEEKLY AND MONTHLY MARKET FEATURES

# merge month number
'''tmp = pd.read_csv("../data/raw/Trade.csv")[["TradeDateKey"]]
tmp = tmp.drop_duplicates()
tmp["TradeDateKey"] = pd.to_datetime(tmp["TradeDateKey"], format = '%Y%m%d')
tmp["Week"] = (tmp.TradeDateKey.dt.year - 2016) * 52 + (tmp.TradeDateKey.dt.week)
tmp["CumMonth"] = (tmp.TradeDateKey.dt.year - 2016) * 12 + (tmp.TradeDateKey.dt.month)
tmp = tmp[["Week", "CumMonth"]]
tmp = tmp.drop_duplicates()
tmp = tmp.groupby("Week").CumMonth.min().reset_index()
data = data.merge(tmp, how = "left", on = "Week")
data["CumMonth"][data.Week == 121] = 28

# import data
market  = pd.read_csv("../data/prepared/data_market_v1.csv", compression = "gzip")
market = market.rename(columns = {"Month": "CumMonth"})
market.CumMonth = market.groupby(["IsinIdx", "Week"]).CumMonth.min()
keep = market[["IsinIdx", "Week", "CumMonth"]].drop_duplicates().index
market = market[market.index.isin(keep)]
data = data.merge(market, how = "left", on = ["IsinIdx", "Week", "CumMonth"])
del data["CumMonth"]
print(data.shape)'''

'tmp = pd.read_csv("../data/raw/Trade.csv")[["TradeDateKey"]]\ntmp = tmp.drop_duplicates()\ntmp["TradeDateKey"] = pd.to_datetime(tmp["TradeDateKey"], format = \'%Y%m%d\')\ntmp["Week"] = (tmp.TradeDateKey.dt.year - 2016) * 52 + (tmp.TradeDateKey.dt.week)\ntmp["CumMonth"] = (tmp.TradeDateKey.dt.year - 2016) * 12 + (tmp.TradeDateKey.dt.month)\ntmp = tmp[["Week", "CumMonth"]]\ntmp = tmp.drop_duplicates()\ntmp = tmp.groupby("Week").CumMonth.min().reset_index()\ndata = data.merge(tmp, how = "left", on = "Week")\ndata["CumMonth"][data.Week == 121] = 28\n\n# import data\nmarket  = pd.read_csv("../data/prepared/data_market_v1.csv", compression = "gzip")\nmarket = market.rename(columns = {"Month": "CumMonth"})\nmarket.CumMonth = market.groupby(["IsinIdx", "Week"]).CumMonth.min()\nkeep = market[["IsinIdx", "Week", "CumMonth"]].drop_duplicates().index\nmarket = market[market.index.isin(keep)]\ndata = data.merge(market, how = "left", on = ["IsinIdx", "Week", "CumMonth"])\ndel data["CumMonth"]\nprin

In [96]:
### FAVORITE FEATURES

bond  = pd.read_csv("../data/raw/Isin.csv")
bond.loc[~bond.Seniority.isin(["GOV","SEC","SEN","SUB"]), "Seniority"] = "OTHER"
bond.loc[~bond.Activity.isin(["Asia","RETAIL","GBP SAS", "ARGENTINIA"]), "Activity"] = "OTHER"
bond.loc[bond.Currency.isin(
    (bond.groupby("Currency").size())[bond.groupby("Currency").size()<350].index.tolist()
), "Currency"] = "OTHER"

bond.loc[bond.CompositeRating.str.contains("C|D"), "CompositeRating"] = "LOWER"
# Risk captain
bond.loc[bond.RiskCaptain.isin(
    (bond.groupby("RiskCaptain").size())[bond.groupby("RiskCaptain").size()<200].index.tolist()
), "RiskCaptain"] = "OTHER"

bond = bond[[
       'IsinIdx', 
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'CouponType']]

favor = pd.read_csv("../data/prepared/favorite_bonds_v2_60.csv", compression = "gzip")

data = data.merge(bond,  how = "left")
data = data.merge(favor, how = "left")

data["BuySell_mode"][data.BuySell == data["BuySell_mode"]] = 1
data["BuySell_mode"][data.BuySell_mode != 1] = 0

data["IsinIdx_mode"][data.IsinIdx == data["IsinIdx_mode"]] = 1
data["IsinIdx_mode"][data.IsinIdx_mode != 1] = 0

data["Seniority_mode"][data.Seniority == data["Seniority_mode"]] = 1
data["Seniority_mode"][data.Seniority_mode != 1] = 0

data["ActivityGroup_mode"][data.ActivityGroup == data["ActivityGroup_mode"]] = 1
data["ActivityGroup_mode"][data.ActivityGroup_mode != 1] = 0

data["Region_mode"][data.Region == data["Region_mode"]] = 1
data["Region_mode"][data.Region_mode != 1] = 0

data["Activity_mode"][data.Activity == data["Activity_mode"]] = 1
data["Activity_mode"][data.Activity_mode != 1] = 0

data["RiskCaptain_mode"][data.RiskCaptain == data["RiskCaptain_mode"]] = 1
data["RiskCaptain_mode"][data.RiskCaptain_mode != 1] = 0

data["Owner_mode"][data.Owner == data["Owner_mode"]] = 1
data["Owner_mode"][data.Owner_mode != 1] = 0

data["CompositeRating_mode"][data.CompositeRating == data["CompositeRating_mode"]] = 1
data["CompositeRating_mode"][data.CompositeRating_mode != 1] = 0

data["Currency_mode"][data.Currency == data["Currency_mode"]] = 1
data["Currency_mode"][data.Currency_mode != 1] = 0

data["CouponType_mode"][data.CouponType == data["CouponType_mode"]] = 1
data["CouponType_mode"][data.CouponType_mode != 1] = 0

excluded_features = ['Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'CouponType']
features = [var for var in data.columns if var not in excluded_features]
data = data[features]

print(data.shape)

(2668912, 65)


### CHECKS AND PREPARATIONS

In [97]:
# check data
print("Dimensions:", data.shape)
data.head()

Dimensions: (2668912, 65)


,PredictionIdx,CustomerIdx,IsinIdx,BuySell,CustomerInterest,Week,Recency1,Recency2,Recency3,Recency4,Recency5,Recency6,Recency1isLowerRecency2,Recency2isLowerRecency4,Frequency1,Frequency2,Frequency3,Frequency4,Frequency5,Frequency6,Frequecny1isLowerFrequency2,Frequecny2isLowerFrequency4,Month,Buy,Asset Managers & Hedge Funds,Asset Owners,Banks and Intermediaries,Corporation,Official Institution - OI,Americas,Asia Pacific,"Europe, Middle East and Africa",FLOW G10,FLOW LOCAL MARKET,SAS & COVERED BONDS,NR,Currency_trend,MeanPrice,StdPrice,MeanYield,StdYield,MeanZScore,StdZScore,YieldMarktDelta,ZScoreMarktDelta,LastWeekCustSum,LastWeekBondSum,LastWeekCustPriceSum,LastWeekCustNotionalSum,LastMonthCustSum,LastMonthBondSum,MaturityWeek,IssueWeek,MaturityPercent,IsinIdx_mode,BuySell_mode,Seniority_mode,Currency_mode,ActivityGroup_mode,Region_mode,Activity_mode,RiskCaptain_mode,Owner_mode,CompositeRating_mode,CouponType_mode
0,NaN,2554,12022,Sell,0.0,68,2.0,2.0,1.0,1.0,2.0,2.0,1,0,0.014925,0.029851,43.716418,80.791045,0.134328,0.208955,1,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1.00000,117.319623,4.613998,4.468827,0.354249,2.207733,0.545221,1.418700,0.584150,27.0,0.0,5.224483e+03,7.224512e+07,173.0,1.0,930,630,-1.873333,0.0,1,1,0,0,0,1,0,0,0,1
1,NaN,1997,23143,Sell,0.0,112,71.0,71.0,1.0,1.0,1.0,1.0,1,0,0.009009,0.009009,4.333333,6.252252,0.990991,2.315315,1,1,3,0,0,1,0,0,0,0,0,1,1,0,0,0,-0.00752,102.526643,8.867004,2.470258,2.011795,2.344190,1.979572,-0.579869,0.720607,0.0,5.0,1.000099e+06,1.400000e+06,14.0,7.0,202,161,-1.195122,0.0,1,1,1,1,1,1,1,0,0,1
2,NaN,2743,5570,Buy,0.0,99,88.0,88.0,1.0,1.0,6.0,4.0,1,0,0.020408,0.020408,62.642857,106.000000,0.775510,1.387755,1,1,5,1,1,0,0,0,0,0,0,1,1,0,0,0,1.00000,102.627270,2.228127,3.838438,0.347182,1.876163,0.581684,0.788311,0.252580,57.0,0.0,9.977553e+03,1.362031e+08,441.0,2.0,359,161,-0.313131,0.0,1,0,1,1,1,1,0,0,0,1
3,NaN,1307,23023,Buy,0.0,91,82.0,82.0,1.0,1.0,20.0,2.0,1,0,0.011111,0.011111,1.288889,2.233333,0.500000,0.977778,1,1,7,1,0,0,1,0,0,0,0,1,1,0,0,0,-0.00294,101.017280,0.204407,-0.036528,0.193049,0.161060,0.150582,-3.086655,-1.462523,1.0,0.0,5.227220e+02,3.768715e+06,2.0,0.0,60,148,0.647727,0.0,1,1,1,1,1,1,0,0,0,1
4,NaN,2107,5204,Sell,0.0,73,2.0,2.0,1.0,1.0,1.0,1.0,1,0,0.027778,0.027778,9.500000,21.541667,0.319444,0.777778,1,1,12,0,1,0,0,0,0,1,0,0,0,1,0,0,1.00000,110.170463,2.443280,5.958544,0.341985,3.637930,0.340257,2.908417,2.014346,3.0,4.0,5.547505e+03,3.071459e+07,27.0,1.0,464,6,0.146288,0.0,0,0,1,0,1,1,0,0,0,1


In [98]:
# check NA
nas = data.isnull().sum()
nas[nas > 0]

PredictionIdx              2184154
CustomerInterest            484758
MeanPrice                   206495
StdPrice                    206601
MeanYield                   206495
StdYield                    206601
MeanZScore                  206495
StdZScore                   206601
YieldMarktDelta             206495
ZScoreMarktDelta            206495
LastWeekCustSum              53095
LastWeekBondSum             439429
LastWeekCustPriceSum        159194
LastWeekCustNotionalSum     159194
LastMonthCustSum             32568
LastMonthBondSum            122039
MaturityPercent                  3
dtype: int64

In [99]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [100]:
col_with_na = ['MeanPrice', 'StdPrice',
       'MeanYield', 'StdYield', 'MeanZScore', 'StdZScore', 'YieldMarktDelta',
       'ZScoreMarktDelta']
col_zero =  ['LastWeekCustSum', 'LastWeekBondSum',
       'LastWeekCustPriceSum', 'LastWeekCustNotionalSum', 'LastMonthCustSum',
       'LastMonthBondSum', 'MaturityPercent']

In [113]:
for col in col_with_na:
    data[col].fillna(np.mean(data[col]), inplace=True)
    
for col in col_zero:
    data[col].fillna(0, inplace=True)

In [114]:
# list of features
excluded_features = ["PredictionIdx", "CustomerIdx", "IsinIdx", "BuySell", "CustomerInterest",
                     "Frequecny1isLowerFrequency2", "Frequecny2isLowerFrequency4"]
features = [var for var in data.columns if var not in excluded_features]
len(features)

58

In [115]:
for var in features:
    
    mmin = data[var].mean() - 5*data[var].std()
    mmax = data[var].mean() + 5*data[var].std()
    
    data[var] = data[var].clip(mmin, mmax)

In [116]:
data[features].describe()

,Week,Recency1,Recency2,Recency3,Recency4,Recency5,Recency6,Recency1isLowerRecency2,Recency2isLowerRecency4,Frequency1,Frequency2,Frequency3,Frequency4,Frequency5,Frequency6,Month,Buy,Asset Managers & Hedge Funds,Asset Owners,Banks and Intermediaries,Corporation,Official Institution - OI,Americas,Asia Pacific,"Europe, Middle East and Africa",FLOW G10,FLOW LOCAL MARKET,SAS & COVERED BONDS,NR,Currency_trend,MeanPrice,StdPrice,MeanYield,StdYield,MeanZScore,StdZScore,YieldMarktDelta,ZScoreMarktDelta,LastWeekCustSum,LastWeekBondSum,LastWeekCustPriceSum,LastWeekCustNotionalSum,LastMonthCustSum,LastMonthBondSum,MaturityWeek,IssueWeek,MaturityPercent,IsinIdx_mode,BuySell_mode,Seniority_mode,Currency_mode,ActivityGroup_mode,Region_mode,Activity_mode,RiskCaptain_mode,Owner_mode,CompositeRating_mode,CouponType_mode
count,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06,2.668912e+06
mean,9.786913e+01,3.718308e+01,2.664048e+01,1.358641e+00,1.160380e+00,5.605520e+00,3.968954e+00,7.607321e-01,8.186632e-02,2.435919e-02,3.914357e-02,4.392110e+01,8.612000e+01,6.498030e-01,1.268837e+00,5.373084e+00,5.112323e-01,6.617363e-01,3.964200e-02,2.784828e-01,1.352629e-04,8.234574e-03,4.158837e-01,5.398042e-02,5.301359e-01,7.117781e-01,2.059041e-01,8.231781e-02,1.000265e-01,5.876176e-01,1.038518e+02,2.384195e+00,3.523718e+00,5.616164e+00,1.748185e+01,6.086593e+01,4.735906e-01,1.585827e+01,7.724105e+01,1.073440e+00,9.070780e+04,2.783914e+08,3.182895e+02,4.235358e+00,3.564957e+02,1.840519e+02,2.944751e-01,5.867062e-04,5.570157e-01,7.513496e-01,8.313848e-01,7.615096e-01,7.417154e-01,9.989068e-01,4.105474e-01,1.729075e-01,1.689823e-01,8.835106e-01
std,1.892735e+01,3.402015e+01,2.582139e+01,1.548490e+00,7.964945e-01,1.058059e+01,8.446211e+00,4.266367e-01,2.741610e-01,3.000516e-02,5.227209e-02,7.173528e+01,1.419835e+02,6.158205e-01,1.189379e+00,3.261264e+00,4.998739e-01,4.731189e-01,1.951167e-01,4.482524e-01,2.813165e-03,6.111244e-02,4.928738e-01,2.259791e-01,4.990911e-01,4.529350e-01,4.043608e-01,2.748484e-01,3.000354e-01,4.918850e-01,7.217017e+00,2.040716e+00,1.649674e+01,9.279880e+01,4.020698e+02,1.453630e+03,1.649674e+01,4.020698e+02,1.425526e+02,1.713911e+00,1.084139e+06,7.523948e+08,5.959914e+02,5.703844e+00,3.855565e+02,1.602801e+02,3.049401e+00,8.457246e-03,4.967387e-01,4.322308e-01,3.744117e-01,4.261606e-01,4.376914e-01,1.348538e-02,4.919333e-01,3.781674e-01,3.747364e-01,3.208110e-01
min,6.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.333333e-03,0.000000e+00,8.333333e-03,0.000000e+00,8.333333e-03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.209600e-01,6.704698e+01,0.000000e+00,-4.876230e+02,0.000000e+00,-9.936235e+01,0.000000e+00,-4.906731e+02,-1.009859e+02,0.000000e+00,0.000000e+00,-6.715030e+06,0.000000e+00,0.000000e+00,0.000000e+00,-1.450000e+02,-4.200000e+01,-2.072693e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.325525e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.200000e+01,9.000000e+00,6.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,9.433962e-03,1.0752

In [117]:
### DATA PARTITIONING

# training
X_train = data.loc[data.Week < 120]
y_train = data.loc[data.Week < 120].CustomerInterest

# validation
X_valid = data.loc[data.Week == 120]
y_valid = data.loc[data.Week == 120].CustomerInterest

# test set
test = data.loc[data.Week == 121]

In [118]:
# check NA in test
nas = test.isnull().sum()
nas[nas > 0]

CustomerInterest    484758
dtype: int64

In [119]:
nas = X_valid.isnull().sum()
nas[nas > 0]

PredictionIdx    40437
dtype: int64

# 4. MODELING - STAGE 1

In [120]:
X_train[features].describe()

,Week,Recency1,Recency2,Recency3,Recency4,Recency5,Recency6,Recency1isLowerRecency2,Recency2isLowerRecency4,Frequency1,Frequency2,Frequency3,Frequency4,Frequency5,Frequency6,Month,Buy,Asset Managers & Hedge Funds,Asset Owners,Banks and Intermediaries,Corporation,Official Institution - OI,Americas,Asia Pacific,"Europe, Middle East and Africa",FLOW G10,FLOW LOCAL MARKET,SAS & COVERED BONDS,NR,Currency_trend,MeanPrice,StdPrice,MeanYield,StdYield,MeanZScore,StdZScore,YieldMarktDelta,ZScoreMarktDelta,LastWeekCustSum,LastWeekBondSum,LastWeekCustPriceSum,LastWeekCustNotionalSum,LastMonthCustSum,LastMonthBondSum,MaturityWeek,IssueWeek,MaturityPercent,IsinIdx_mode,BuySell_mode,Seniority_mode,Currency_mode,ActivityGroup_mode,Region_mode,Activity_mode,RiskCaptain_mode,Owner_mode,CompositeRating_mode,CouponType_mode
count,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06,2.143717e+06
mean,9.222109e+01,3.369933e+01,2.982196e+01,1.353249e+00,1.165033e+00,6.002391e+00,4.377038e+00,8.174069e-01,8.424573e-02,2.681437e-02,4.186655e-02,4.882405e+01,9.553752e+01,7.102198e-01,1.382632e+00,5.766068e+00,5.140525e-01,6.645877e-01,3.719008e-02,2.779173e-01,1.350001e-04,8.313299e-03,4.303045e-01,5.319219e-02,5.165033e-01,7.128194e-01,2.062670e-01,8.091367e-02,9.893237e-02,5.966740e-01,1.039965e+02,2.435175e+00,3.646985e+00,6.336522e+00,2.013491e+01,7.086485e+01,5.968582e-01,1.851133e+01,8.163099e+01,1.087682e+00,7.432801e+04,3.040406e+08,3.367044e+02,4.313704e+00,3.563241e+02,1.887799e+02,2.889688e-01,6.779315e-04,5.713198e-01,7.522817e-01,8.375868e-01,7.660083e-01,7.475404e-01,9.992521e-01,4.162574e-01,1.773900e-01,1.715586e-01,8.875724e-01
std,1.684889e+01,2.806210e+01,2.704833e+01,1.524659e+00,8.067271e-01,1.114241e+01,9.154716e+00,3.863327e-01,2.777560e-01,3.121955e-02,5.496171e-02,7.601550e+01,1.505329e+02,6.387655e-01,1.233831e+00,3.507142e+00,4.998026e-01,4.721345e-01,1.892274e-01,4.479725e-01,2.810438e-03,6.139854e-02,4.951188e-01,2.244166e-01,4.997277e-01,4.524467e-01,4.046245e-01,2.727025e-01,2.985713e-01,4.911850e-01,7.245019e+00,2.050197e+00,1.764138e+01,9.864885e+01,4.310445e+02,1.559436e+03,1.764138e+01,4.310445e+02,1.503673e+02,1.768565e+00,9.533069e+05,8.094768e+08,6.292594e+02,5.958899e+00,3.905590e+02,1.615228e+02,3.097731e+00,9.087595e-03,4.948875e-01,4.316874e-01,3.688295e-01,4.233671e-01,4.344236e-01,1.116586e-02,4.929374e-01,3.819985e-01,3.769964e-01,3.158919e-01
min,6.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,8.474576e-03,8.474576e-03,8.474576e-03,8.474576e-03,8.474576e-03,8.474576e-03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.209600e-01,6.704698e+01,0.000000e+00,-4.876230e+02,0.000000e+00,-9.936235e+01,0.000000e+00,-4.906731e+02,-1.009859e+02,0.000000e+00,0.000000e+00,-6.715030e+06,0.000000e+00,0.000000e+00,0.000000e+00,-1.450000e+02,-4.200000e+01,-2.072693e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.325525e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.700000e+01,9.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.041667e-02,1.1764

In [121]:
from sklearn.linear_model import LogisticRegression

In [122]:
nb = LogisticRegression(C=1.0)

In [123]:
nb.fit(X_train[features], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [124]:
pred_valid = X_valid[["CustomerIdx", "IsinIdx", "BuySell", "Week", "CustomerInterest"]]
pred_valid["TARGET"] = nb.predict_proba(X_valid[features])[:,1]
auc = roc_auc_score(y_valid, pred_valid.TARGET)
auc

0.3483107920240308

In [82]:
##### PREDICTION

### VALID DATA

# predict validation set
pred_valid = X_valid[["CustomerIdx", "IsinIdx", "BuySell", "Week", "CustomerInterest"]]
pred_valid["TARGET"] = nb.predict_proba(X_valid[features])[:,1]
auc = roc_auc_score(y_valid, pred_valid.TARGET)

# export CSV
pred_valid.to_csv("../pred_valid/auc" + str(round(auc, 6))[2:8] + "_" + str(data_name) + "_nb.csv", 
                  index = False, float_format = "%.8f")


### TEST DATA

# predict test set
test["TARGET"] = nb.predict_proba(test[features])[:,1]

# export CSV
subm = test[["PredictionIdx", "TARGET"]]
subm.columns = ["PredictionIdx", "CustomerInterest"]
subm.to_csv("../submissions/auc" + str(round(auc, 6))[2:8] + "_" + str(data_name) + "_nb_1stage.csv", 
            index = False, float_format = "%.8f")

In [83]:
auc

0.6255990207875272

# 5. MODELING - STAGE 2

In [84]:
# use full data as train
X_train = data.loc[data.Week <= 120]
y_train = data.loc[data.Week <= 120].CustomerInterest

In [85]:
nb = GaussianNB()
nb.fit(X_train[features], y_train)

GaussianNB(priors=None)

In [86]:
# export CSV
subm = test[["PredictionIdx", "TARGET"]]
subm.columns = ["PredictionIdx", "CustomerInterest"]
subm.to_csv("../submissions/auc" + str(round(auc, 6))[2:8] + "_" + str(data_name) + "_nb_2stage.csv", 
            index = False, float_format = "%.8f")